<a href="https://colab.research.google.com/github/lijin-durairaj-code-mode/codes/blob/main/RAG_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install libraries

In [10]:
# !pip install langchain_community
# !pip install langchain_core
# !pip install langchain
# !pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.4 MB/s eta 0:00:00


## import libraries

In [2]:
#langchain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_core.documents import Document

#hhugging face
from huggingface_hub import HfFolder

#google
from google.colab import userdata

#common libraries
import pandas as pd
import json

In [35]:
import faiss

### colab token

In [3]:
HfFolder.save_token(userdata.get('hf_token'))

# trail & error

In [5]:
from langchain import hub
rag_prompt=hub.pull('rlm/rag-prompt')

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# end

#### process document

In [4]:
def format_data(r):
  _content=f'''{r['Salutation']} {r['Initial']} {r['Name']}, Employee Id is {r['Employee Id']} and user id is {r['User Id']},  is from country {r['Country']},  Supervisor / Manager name is {r['Supervisor / Manager']}  and Contracting Company is  {r['Contracting Company']},  Primary Industry is {r['Primary Industry']}  and Secondary Industry	is {r['Secondary Industry']}, there Sector is {r['Sector']}, they are expertise in {r['Expertise']}, there role is {r['Industry Role']}. There last Last Promotion Date is  {r['Last Promotion Date']} and {r['Last Promotional Level']}, There Job Title is {r['Job Title']}, they are working here since {r['Professional Since']}, there hired date is {r['Hired Date']}, there Relevant is {r['Relevant']}, Employee Sponser is {r['Employee Sponser']}, Job Description is {r['Job Description']},  Emergency Contact Name is {r['Emergency Contact Name']} and Emergency Contact Number  is {r['Emergency Contact Number']}, Regional Supervisor is {r['Regional Supervisor']}  Office Supervisor is {r['Office Supervisor']} Engagement Supervisor is {r['Engagement Supervisor']} '''

  _meta={
              "Salutation":r["Salutation"],
              "Initial":r["Initial"],
              "User Id" :r["User Id"],
              "Name" :r["Name"],
              "Employee Id" :r["Employee Id"],
              "Country":r["Country"],
              "Supervisor / Manager" :r["Supervisor / Manager"],
              "Contracting Company" :r["Contracting Company"],
              "Primary Industry":r["Primary Industry"],
              "Secondary Industry" :r["Secondary Industry"],
              "Sector" :r["Sector"],
              "Expertise":r["Expertise"],
              "Industry Role":r["Industry Role"],
              "Designation" :r["Designation"],
              "Grade" :r["Grade"],
              "Target Chargeability %" :r["Target Chargeability %"],
              "Charge Out Rate":r["Charge Out Rate"],
              "Last Promotion Date" :r["Last Promotion Date"],
              "Last Promotional Level" :r["Last Promotional Level"],
              "Job Title":r["Job Title"],
              "Professional Since" :r["Professional Since"],
              "Hired Date" :r["Hired Date"],
              "Relevant":r["Relevant"],
              "Employee Sponser":r["Employee Sponser"],
              "Job Description" :r["Job Description"],
              "Emergency Contact Name" :r["Emergency Contact Name"],
              "Emergency Contact Number":r["Emergency Contact Number"],
              "Regional Supervisor" :r["Regional Supervisor"],
              "Office Supervisor" :r["Office Supervisor"],
              "Engagement Supervisor":r["Engagement Supervisor"]
          }
  return _content,_meta

In [5]:
document_list=[]
employee_data=pd.read_csv('employee-list.csv')
for e in json.loads(str(employee_data.to_json(orient='records'))):
  d,m=format_data(e)
  document=Document(
      page_content=d,
      metadata=m
  )
  document_list.append(document)

In [8]:
document_list[0]

Document(metadata={'Salutation': 'Prof.', 'Initial': 'R', 'User Id': 'aaradhya@gmail.com', 'Name': 'Anjana Raj', 'Employee Id': 'WA7898123', 'Country': 'Rwanda', 'Supervisor / Manager': None, 'Contracting Company': 'BDO Africa Holdings Ltd', 'Primary Industry': 'Technologies, Media and Telecom', 'Secondary Industry': None, 'Sector': None, 'Expertise': 'Development', 'Industry Role': None, 'Designation': 'develop', 'Grade': 'P2', 'Target Chargeability %': 10.0, 'Charge Out Rate': 50.0, 'Last Promotion Date': None, 'Last Promotional Level': None, 'Job Title': 'Developer', 'Professional Since': '13-05-2024', 'Hired Date': '13-05-2024', 'Relevant': None, 'Employee Sponser': 'yes', 'Job Description': None, 'Emergency Contact Name': 'Guhan', 'Emergency Contact Number': 91451198306.0, 'Regional Supervisor': 'Akass Akash', 'Office Supervisor': 'Ajay S', 'Engagement Supervisor': 'Anj Anj'}, page_content='Prof. R Anjana Raj, Employee Id is WA7898123 and user id is aaradhya@gmail.com,  is from co

#### set FAISS & vectorstore

In [9]:
model_path='sentence-transformers/all-MiniLM-L12-v2'
model_kwargs={
    'device':'cuda'
}
encode_kwargs={
    'normalize_embeddings':False
}

embeddings=HuggingFaceEmbeddings(
    model_name=model_path,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

<ipython-input-9-7097619cfbe6>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [11]:
vector_db=FAISS.from_documents(document_list,embeddings)
vector_db.save_local('faiss_employee_list_index')
retriver=vector_db.as_retriever()

# trail

In [24]:
texts = [
    "Artificial intelligence is the future.",
    "Machine learning is a subset of AI.",
    "Natural language processing enables interaction with computers."
]

text_list=[]
for t in texts:
  c=f'''{t}'''
  document=Document(
      page_content=c
  )
  text_list.append(document)


text_list

[Document(metadata={}, page_content='Artificial intelligence is the future.'),
 Document(metadata={}, page_content='Machine learning is a subset of AI.'),
 Document(metadata={}, page_content='Natural language processing enables interaction with computers.')]

In [28]:
vector_embeds=embeddings.embed_documents(texts)

In [51]:
embeddings_arr=np.array(vector_embeds).astype('float32')

In [38]:
embeddings_arr.shape[1]

384

In [39]:
index=faiss.IndexFlatL2(384)

In [40]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7fbc0c78db00> >

In [50]:
type(embeddings_arr[0][1])

numpy.float64

In [52]:
index.add(embeddings_arr)

In [53]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x7fbc0c78db00> >

In [54]:
query_vector=embeddings.embed_query('what is machine learning!')


In [60]:
d,i=index.search(np.array([query_vector]).astype('float32'),2)

In [67]:
for _i in range(2):
  print(f'distance:{d[-1][_i]} text:{texts[i[-1][_i]]}')

distance:0.7361218333244324 text:Machine learning is a subset of AI.
distance:1.0868109464645386 text:Artificial intelligence is the future.


In [27]:
import numpy as np
embedding_matrix=np.array(vector_embed).astype('float32')

TypeError: float() argument must be a string or a real number, not 'FAISS'

# end

In [14]:
retriver.invoke('saran')

[Document(metadata={'Salutation': 'Dr.', 'Initial': 'S', 'User Id': 'saran@gmail.com', 'Name': 'Saran Developer', 'Employee Id': '12344', 'Country': 'Kenya', 'Supervisor / Manager': None, 'Contracting Company': None, 'Primary Industry': None, 'Secondary Industry': None, 'Sector': None, 'Expertise': None, 'Industry Role': None, 'Designation': 'Tester', 'Grade': 'A1', 'Target Chargeability %': None, 'Charge Out Rate': None, 'Last Promotion Date': None, 'Last Promotional Level': None, 'Job Title': 'Tester', 'Professional Since': '07-01-2013', 'Hired Date': '09-05-2024', 'Relevant': None, 'Employee Sponser': 'no', 'Job Description': None, 'Emergency Contact Name': None, 'Emergency Contact Number': None, 'Regional Supervisor': 'Ajay S', 'Office Supervisor': 'Akass Akash', 'Engagement Supervisor': None}, page_content='Dr. S Saran Developer, Employee Id is 12344 and user id is saran@gmail.com,  is from country Kenya,  Supervisor / Manager name is None  and Contracting Company is  None,  Prima

#### format document

#### declare methods